In [1]:
!pip install dnspython
!pip install ipynb

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
import requests
import dns.resolver
# Import all functions from Funtions.ipynb
from ipynb.fs.full.Functions import *

#PostgreSQL access data
host = "bda_gr4_database"
port = "5432"
database = "domainanalysis"
user = "postgres"
password = "postgres"

# PostgreSQL connection url
connection = f"jdbc:postgresql://{host}:{port}/{database}"

# Create a Spark session
spark = SparkSession.builder \
    .appName("domain_analysis") \
    .getOrCreate()

# Read data from the database
domains_df = spark.read \
                .format("jdbc") \
                .option("url", connection) \
                .option("dbtable", "domain") \
                .option("user", user) \
                .option("password", password) \
                .load()

In [17]:
udf_getIPv6Records = udf(getIPv6Records, ArrayType(StringType()))
udf_getSOAInformation = udf(getSOAInformation, ArrayType(StringType()))
udf_getSOAInformation_error = udf(getSOAInformation_error, ArrayType(StringType()))
udf_getNameServers = udf(getNameServers, ArrayType(StringType()))
udf_getNameServers_error = udf(getNameServers_error, IntegerType())

In [21]:
domains_df = domains_df.withColumn("aaaa_record", udf_getIPv6Records("top_level_domain"))
domains_df = domains_df.withColumn("soa_information", udf_getSOAInformation("top_level_domain"))
domains_df = domains_df.withColumn("soa_information_error", udf_getSOAInformation_error("top_level_domain"))
domains_df = domains_df.withColumn("nameservers", udf_getNameServers("top_level_domain"))
domains_df = domains_df.withColumn("nameservers_error", udf_getNameServers_error("top_level_domain"))
domains_df.show(20, False)

AttributeError: 'NoneType' object has no attribute 'withColumn'

In [4]:
# Change ArrayType<String> into String as preparation for information separation
domains_df = domains_df.withColumn("soa_infos_rep", concat_ws(" ", "soa_information"))

# Split SOA information into separate columns (all String)
split_col = split(domains_df['soa_infos_rep'], ' ')
domains_df = domains_df.withColumn('mname', split_col.getItem(0))
domains_df = domains_df.withColumn('refresh', split_col.getItem(3))
domains_df = domains_df.withColumn('minimum', split_col.getItem(6))

#
def replace_empty_strings(x):
    return when(col(x) == "", None).otherwise(col(x))

domains_df = domains_df.withColumn("mname", replace_empty_strings("mname"))

In [5]:
# Remove last dot per soa mname
domains_df = domains_df.withColumn('mname', regexp_replace('mname', '.$', ''))   

# Remove last dot per nameserver entry
lambda_dot_remove = lambda arr: [x[:-1] for x in arr]
def fn_remove_dot(arr): return None if arr == None else lambda_dot_remove(arr)
udf_remove_last_char_in_array = udf(fn_remove_dot, ArrayType(StringType()))

domains_df = domains_df \
    .select("*") \
    .withColumn('nameservers', udf_remove_last_char_in_array(col('nameservers')))

In [13]:
# change dtypes of seconds to int
domains_df = domains_df.withColumn("refresh", domains_df["refresh"].cast(IntegerType()))
domains_df = domains_df.withColumn("minimum", domains_df["minimum"].cast(IntegerType()))

#domains_df.select('mname').show(20, False)
domains_df.show(5)

+----------------+--------------------+-----------------+--------------------+--------------------+---------------------+--------------------+-----------------+--------------------+--------------------+-------+-------+
|top_level_domain|           mx_record|         a_record|         aaaa_record|     soa_information|soa_information_error|         nameservers|nameservers_error|       soa_infos_rep|               mname|refresh|minimum|
+----------------+--------------------+-----------------+--------------------+--------------------+---------------------+--------------------+-----------------+--------------------+--------------------+-------+-------+
|          0-3.de|[teller.ggeg.eu, ...|[185.163.116.240]|                null|[teller.ggeg.eu. ...|                 null|[teller.ggeg.eu, ...|                0|teller.ggeg.eu. e...|      teller.ggeg.eu|  86400|   3600|
|     0-263475.de|                null|   [176.9.76.101]|                null|[brit.ns.cloudfla...|                 null|[br

In [7]:
# Count the values of each column with descending sort and show it
# domains_df.groupBy("Top-Level-Domain").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("MX-Record").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("A-Record").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("A-Record_Checked").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("MX-Record_Checked").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("Redirection").count().sort(desc("count")).show(20, False)

In [8]:
# Count the occrence of SOA records
soa_mname_count = domains_df.withColumn('mname', (col('mname'))) \
        .groupBy('mname') \
        .count()

soa_mname_count_top_ten_df = soa_mname_count.orderBy(['count'], ascending = [False]).limit(10)
#soa_mname_count_top_ten_df.show()

In [9]:
def getSOAIPv4(mname):
    if mname == None:
        return None
    resolver = dns.resolver.Resolver(); 
    response = resolver.query(mname , "A")
    soa_ipv4 = ''
    for item in response:
        ip_string = ','.join([str(item), soa_ipv4])
        ip_string = ip_string[:len(ip_string)-1]
    return ip_string

In [10]:
udf_getSOAIPv4 = udf(getSOAIPv4, StringType())
soa_mname_count_top_ten_df = soa_mname_count_top_ten_df.withColumn("ipv4", udf_getSOAIPv4("mname"))
#soa_mname_count_top_ten_df.show(5, False)

In [11]:
# ===================REMOVE AND DELETE ?
udf_getOrganisation = udf(getOrg, StringType())
udf_getPostal = udf(getPostal, StringType())
udf_getCity = udf(getCity, StringType())
udf_getCountry = udf(getCountry, StringType())

In [12]:
# Add company information to SOA server
soa_mname_count_top_ten_df = soa_mname_count_top_ten_df.withColumn("organization", udf_getOrganisation("ipv4"))
soa_mname_count_top_ten_df = soa_mname_count_top_ten_df.withColumn("postal", udf_getPostal("ipv4"))
soa_mname_count_top_ten_df = soa_mname_count_top_ten_df.withColumn("city", udf_getCity("ipv4"))
soa_mname_count_top_ten_df = soa_mname_count_top_ten_df.withColumn("country", udf_getCountry("ipv4"))
soa_mname_count_top_ten_df.show(5, False)

+---------------------+-----+---------------+--------------------------+------+-----------------+-------+
|mname                |count|ipv4           |organization              |postal|city             |country|
+---------------------+-----+---------------+--------------------------+------+-----------------+-------+
|ns1.sedoparking.com  |6    |91.195.241.8   |AS47846 SEDO GmbH         |60311 |Frankfurt am Main|DE     |
|null                 |4    |null           |null                      |null  |null             |null   |
|dns01-tld.t-online.de|3    |212.185.250.107|AS3320 Deutsche Telekom AG|50676 |Köln             |DE     |
|root-dns.netcup.net  |2    |46.38.225.225  |AS197540 netcup GmbH      |76461 |Muggensturm      |DE     |
|ns1.hosting.de       |1    |134.0.30.178   |AS48823 Hosting.de GmbH   |33519 |Bielefeld        |DE     |
+---------------------+-----+---------------+--------------------------+------+-----------------+-------+
only showing top 5 rows



In [15]:
# # domains_df = domains_df.withColumn('mx_record_count', explode(col('mx_record'))) \
# #         .groupBy('mx_record_count') \
# #         .count() \
# #         .sort(desc("count"))

# domains_df = domains_df.withColumn('mx_record_checked_count', explode(col('mx_record_checked'))) \
#         .groupBy('mx_record_checked_count') \
#         .count() \
#         .sort(desc("count"))

# # domains_df = domains_df.withColumn('a_record_count', explode(col('a_record'))) \
# #         .groupBy('a_record_count') \
# #         .count() \
# #         .sort(desc("count"))

# domains_df = domains_df.withColumn('a_record_checked', explode(col('a_record_checked'))) \
#         .groupBy('a_record_checked') \
#         .count() \
#         .sort(desc("count"))

# domains_df = domains_df.withColumn('m_name', explode(col('m_name'))) \
#         .groupBy('m_name') \
#         .count() \
#         .sort(desc("count"))

# domains_df = domains_df.withColumn('nameservers_count', explode(col('nameservers'))) \
#         .groupBy('nameservers_count') \
#         .count() \
#         .sort(desc("count"))

domains_df = domains_df.drop('mx_record').drop('a_record').drop('soa_information').drop('soa_infos_rep')

In [16]:
domains_df.show(20)

+----------------+--------------------+---------------------+--------------------+-----------------+--------------------+-------+-------+
|top_level_domain|         aaaa_record|soa_information_error|         nameservers|nameservers_error|               mname|refresh|minimum|
+----------------+--------------------+---------------------+--------------------+-----------------+--------------------+-------+-------+
|          0-3.de|                null|                 null|[teller.ggeg.eu, ...|                0|      teller.ggeg.eu|  86400|   3600|
|     0-263475.de|                null|                 null|[brit.ns.cloudfla...|                0|brit.ns.cloudflar...|  10000|   3600|
|       0-apps.de|                null|                 null|[robotns3.second-...|                0|     ns1.first-ns.de|  14400|  86400|
|         0-32.de|[2a01:238:20a:202...|                 null|[docks12.rzone.de...|                0|   shades10.rzone.de|  86400|    300|
|         0-17.de|                

In [ ]:
# Write the data frame to the PostgreSQL database
domains_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_enhanced") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()